In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
#This file contains code to train the model on input music files


import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

def get_notes():
    """ Get all the notes and chords from the midi files in the ./input_music directory """
    notes = []

    for file in glob.glob("/content/drive/My Drive/Nokia/input_music/*.mid"):
        try:
            midi = converter.parse(file)
        except:
            print(" this is a")
            continue

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', 
                  optimizer='rmsprop', 
                  metrics=['accuracy'])

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "trained_models/weights-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    history = model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
    return history

if __name__ == '__main__':
    
    # Pre processing of data
    notes = get_notes()
    n_vocab = len(set(notes))
    
    # Data formation
    network_input, network_output = prepare_sequences(notes, n_vocab)
    # Model architecture
    model = create_network(network_input, n_vocab)
    # Treaining
    history = train(model, network_input, network_output)
    # Ploting


Parsing /content/drive/My Drive/Nokia/input_music/23.mid
Parsing /content/drive/My Drive/Nokia/input_music/6.mid
Parsing /content/drive/My Drive/Nokia/input_music/54.mid
Parsing /content/drive/My Drive/Nokia/input_music/43.mid
Parsing /content/drive/My Drive/Nokia/input_music/89.mid
Parsing /content/drive/My Drive/Nokia/input_music/40.mid
Parsing /content/drive/My Drive/Nokia/input_music/21.mid
Parsing /content/drive/My Drive/Nokia/input_music/46.mid
Parsing /content/drive/My Drive/Nokia/input_music/72.mid
Parsing /content/drive/My Drive/Nokia/input_music/30.mid
Parsing /content/drive/My Drive/Nokia/input_music/73.mid
Parsing /content/drive/My Drive/Nokia/input_music/61.mid
Parsing /content/drive/My Drive/Nokia/input_music/29.mid
Parsing /content/drive/My Drive/Nokia/input_music/58.mid
Parsing /content/drive/My Drive/Nokia/input_music/2.mid
Parsing /content/drive/My Drive/Nokia/input_music/5.mid
Parsing /content/drive/My Drive/Nokia/input_music/19.mid
Parsing /content/drive/My Drive/No